In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import time
import glob

try:
    import h5py
except:
    !sudo pip install h5py

import sys
PATH_TO_CODE = '../code/'
sys.path.append(os.path.abspath(PATH_TO_CODE))

from kinematics_decoding import *

**Load Dataset.**

In [ ]:
# Load data and select 200 trajectories of each class
with h5py.File('../dataset/pcr_dataset_train.hdf5', 'r') as dataset:
    labels = dataset['label'][()]

In [ ]:
np.random.seed(1219)
num_traj = 200
idx_set = []
for char_id in range(1, 21):
    char_idx = np.argwhere(labels == char_id).flatten()
    char_selected_idx = np.random.choice(char_idx, num_traj)
    idx_set.append(char_selected_idx)
    
idx_set = np.asarray(idx_set).flatten()

In [ ]:
with h5py.File('../dataset/pcr_dataset_train.hdf5', 'r') as dataset:
    spindle_firing = dataset['spindle_firing'][()][idx_set]
    muscle_coords = dataset['muscle_coords'][()][idx_set]
    joint_coords = dataset['joint_coords'][()][idx_set]
    endeff_coords = dataset['endeffector_coords'][()][idx_set]

In [ ]:
spindle_rdm = compute_rdm(spindle_firing)
muscle_rdm = compute_rdm(muscle_coords)
joint_rdm = compute_rdm(joint_coords)
endeff_rdm = compute_rdm(endeff_coords)

In [ ]:
results = {'spindle_firing': spindle_rdm,
           'muscle_coords': muscle_rdm,
           'endeffector_coords': endeff_rdm,
           'joint_coords': joint_rdm}
           
pickle.dump(results, open('kinematics_rdms.p', 'wb'))

**Load Models, Generate Representations, RDMs and save.**

In [ ]:
all_models = pickle.load(open('../nn-training/analyzed_convmodels.p', 'rb'))

In [ ]:
for i in range(all_models.shape[0]):
    mymodel = load_model(all_models.iloc[i], 4, 'conv', all_models['is_trained'].iloc[i])
    generate_layerwise_representations(mymodel, spindle_firing, 200, 'rdm')

    with open('progress.txt', 'a+') as f:
        f.write(f'Finished generating representations for model {i}/20 \n')
        f.write(f'Time: {time.ctime()}\n')

    layerlist = glob.glob(os.path.join(mymodel.model_path, 'rdm_*'))
    model_results = {}
    for layerfile in layerlist:
        layername = os.path.basename(layerfile)[4:-5]
        layer_rdm = compute_rdm(get_representations(layerfile))        
        model_results[layername] = layer_rdm
        os.remove(layerfile)

    with open('progress.txt', 'a+') as f:
        f.write(f'Finished generating RDMs for model {i}/20 \n')
        f.write(f'Time: {time.ctime()}\n')

    results_file = os.path.join(mymodel.model_path, 'model_rdm.p')
    pickle.dump(model_results, open(results_file, 'wb'))